In [3]:
import pandas as pd 
import numpy as np 
from create_df import load_df, implement_column_creation
pd.set_option('display.max_columns', None)
df = load_df()
df = implement_column_creation(df)
df

/home/sam/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/home/sam/Documents/DSI/capstone/poker/src/datacleaning/dependent_columns.py:245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prior_actions'] = df.HandHistory.apply(lambda x: make_lst(x))
/home/sam/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/sam/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:

,HandHistoryId,HandNumber,PokerSiteId,HandHistory,HandHistoryTimestamp,GameType,TournamentNumber,buyin,cash_or_tourn,my_blind_anti_total,my_cards,starting_stack,tournament_type,won,bet,total_players,position,table_max_players,bets_before_my_preflop_action,action_type,player_names,players_acting_before_me,players_acting_after_me,limpers,raises&reraises,callers,hour,year_month_day,days_since_start,hand_frequency,blinds,made_money,card_rank,BB,BB_in_stack,net_outcome,all_in,money_beyond_blind,outcome_relative_to_start,made_or_lost,prior_actions,vpip_all_players,vpip_players_after,vpip_players_before,num_players_before,num_players_after,num_actions_before_players,num_actions_after_players,amount_to_call,average_table_vpip,total_actions_witnessed,vpip_relavant_players,total_actions_witnessed_relevant_players,weighted_relevant_vpip,flop_bet,turn_bet,river_bet,preflop_bet,high_card,low_card,suited,pocket_pair,gap
0,1,3951210633,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:08:08,1,459742,1.10,Holdem NL,20.0,"[C5, H2]",1500.0,3-Table Sit & Go,0.0,20.0,9,0.222222,9,80.0,"{'P26_251842VD': 3, 'P4_131206CX': 0, 'P11_924...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_...","[P26_251842VD, P25_871309MO]",{},1,1,0,21,2019-10-23,0,2155,"[10.0, 20.0]",0,0.5,20.0,75.00,-20.0,0.0,0,0.986667,0,"{'P18_572249IY': [], 'P14_459713MR': [], 'P25_...","{'P18_572249IY': 'No Hands', 'P14_459713MR': '...",NaN,NaN,2,0,0,0,2.0,NaN,0,NaN,0,NaN,0.0,0.0,0.0,0.0,5.0,2.0,0,0,3.0
1,2,3951211988,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:09:34,1,459742,1.10,Holdem NL,10.0,"[C9, S9]",1480.0,3-Table Sit & Go,640.0,420.0,9,0.111111,9,40.0,"{'P4_131206CX': 0, 'P11_924370QJ': 0, 'P18_572...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_...","[P14_459713MR, P25_871309MO]",{P26_251842VD},2,0,0,21,2019-10-23,0,2155,"[10.0, 20.0]",1,9.0,20.0,74.00,220.0,0.0,1,1.148649,1,"{'P18_572249IY': [0], 'P14_459713MR': [0], 'P2...","{'P18_572249IY': 0.0, 'P14_459713MR': 0.0, 'P2...",1.000000,0.500000,2,1,2,1,0.5,0.250000,8,0.666667,3,0.666667,320.0,0.0,0.0,90.0,9.0,9.0,0,1,0.0
2,3,3951212830,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:11:47,1,459742,1.10,Holdem NL,0.0,"[S3, DK]",1700.0,3-Table Sit & Go,0.0,0.0,9,1.000000,9,200.0,"{'P11_924370QJ': 0, 'P18_572249IY': 0, 'P14_45...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_...","[P14_459713MR, P5_405793KB]","{P4_131206CX, P26_251842VD}",0,1,1,21,2019-10-23,0,2155,"[10.0, 20.0]",0,3.0,20.0,85.00,0.0,0.0,0,1.000000,1,"{'P18_572249IY': [0, 0], 'P14_459713MR': [0, 3...","{'P18_572249IY': 0.0, 'P14_459713MR': 0.5, 'P2...",0.250000,0.250000,2,2,4,4,5.0,0.250000,16,0.250000,8,0.250000,0.0,0.0,0.0,0.0,13.0,3.0,0,0,10.0
3,4,3951214131,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:13:37,1,459742,1.10,Holdem NL,0.0,"[C9, D8]",1700.0,3-Table Sit & Go,0.0,0.0,8,0.875000,9,0.0,"{'P18_572249IY': 0, 'P25_871309MO': 0, 'P3_286...","[P18_572249IY, P25_871309MO, P3_286963JX, P5_4...",[],"{P4_131206CX, P11_924370QJ, P26_251842VD}",0,0,0,21,2019-10-23,0,2155,"[10.0, 20.0]",0,5.5,20.0,85.00,0.0,0.0,0,1.000000,1,"{'P18_572249IY': [0, 0, 0], 'P25_871309MO': [2...","{'P18_572249IY': 0.0, 'P25_871309MO': 0.666666...",0.111111,NaN,0,3,0,9,1.0,0.190476,21,0.111111,9,0.111111,0.0,0.0,0.0,0.0,9.0,8.0,0,0,1.0
4,5,3951215154,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:14:31,1,459742,1.10,Holdem NL,0.0,"[S7, HQ]",1700.0,3-Table Sit & Go,0.0,0.0,8,0.750000,9,0.0,"{'P25_871309MO': 0, 'P3_286963JX': 0, 'P5_4057...","[P18_572249IY, P25_871309MO, P3_286963JX, P5_4...",[P5_405793KB],"{P18_572249IY, P4_131206CX, P11_924370QJ, P26_...",0,1,0,21,2019-10-23,0,2155,"[10.0, 20.0]",0,2.0,20.0,85.00,0.0,0.0,0,1.000000,1,"{'P18_572249IY': [0, 0, 0, 0], 'P25_871309MO':...","{'P18_572249IY': 0.0, 'P25_871309MO': 0.5, 'P3...",0.187500,0.250000,1,4,4,16,5.0,0.214286,28,0.200000,20,0.200000,0.0,0.0,0.0,0.0,12.0,7.0,0,0,5.0
...,...,...,...,...,...,...,...

In [6]:
df.num_players_after.value_counts()

0    10762
1     7453
2     6679
3     5352
4     4070
5     2760
6     1257
7      877
8      341
Name: num_players_after, dtype: int64

In [9]:
def actions_witnessed_after(row):
    prior_actions = row['prior_actions']
    total = 0
    for player in row['players_acting_after_me']:
        total += len(prior_actions[player])
    return total 
df['num_actions_after_players'] = df.apply(lambda row: actions_witnessed_after(row), axis=1)

In [4]:
def average_table_vpip(row):
    vpip_all = row['vpip_all_players']
    vpip_lst = []
    players_used = 0
    for key, val in vpip_all.items():
        if key == 'Hero' or val == 'No Hands':
            continue
        else:
            vpip_lst.append(val)
            players_used += 1
    return np.mean(vpip_lst)

In [11]:

def vpip_relavant_players(row):
    vpip_lst = []
    players_before = row['players_acting_before_me']
    players_after = row['players_acting_after_me']
    vpip_dic = row['vpip_all_players']
    for player in players_before:
        if vpip_dic[player] != 'No Hands':
            vpip_lst.append(vpip_dic[player])
    for player in players_after:
        if vpip_dic[player] != 'No Hands':
            vpip_lst.append(vpip_dic[player])
        
    return np.mean(vpip_lst)
    


In [17]:
df.HandHistory.iloc[1000]

['<session sessioncode="1180056">',
 '<general>',
 '<mode>real</mode>',
 '<gametype>Holdem NL</gametype>',
 '<tablename>$500 Guaranteed ($2 Knockout 10K Chips Turbo) (#30201467)</tablename>',
 '<duration>N/A</duration>',
 '<gamecount>0</gamecount>',
 '<startdate>2019-10-30 02:37:50</startdate>',
 '<currency>USD</currency>',
 '<nickname>Hero</nickname>',
 '<bets>0</bets>',
 '<wins>0</wins>',
 '<chipsin>0</chipsin>',
 '<chipsout>0</chipsout>',
 '<ipoints>0</ipoints>',
 '<statuspoints>0</statuspoints>',
 '<awardpoints>0</awardpoints>',
 '<tournamentname>$500 Guaranteed ($2 Knockout 10K Chips Turbo) (#30201467)</tournamentname>',
 '<place>51</place>',
 '<buyin>$7+$0.70</buyin>',
 '<totalbuyin>$7.70</totalbuyin>',
 '<win>0</win>',
 '<maxplayers>9</maxplayers>',
 '<pokersite>Ignition</pokersite>',
 '</general>',
 '<game gamecode="3953871497">',
 '<general>',
 '<startdate>2019-10-30 04:17:38</startdate>',
 '<players>',
 '<player seat="6" name="P64_513163EY" chips="51861" dealer="0" win="0" be

In [55]:
def callers(x): # pretty sure this works but consider checking to make sure 
    bet = 0
    player_name = 0
    player_amount_bet = {} # store amount bet to delineate shoves from calls (all shoves are shoves but some are raises and some are calls)       
    for elem in x: # getting amount put in prior to betting
        if '[cards]' in elem and ('type="1"' in elem or 'type="2"' in elem): # ignores antis  
            for sub_string in elem.split():
                if 'player' in sub_string:
                    player_name = sub_string.replace('player="', '').replace('"', '') 
                if 'sum' in sub_string:
                    bet = float(sub_string.replace('sum="', '').replace('"', '')) 
                player_amount_bet[player_name] = bet
                            
    for idx, elem in enumerate(x): 
        if 'Pocket' in elem:
            start = idx
    subset = x[start+1:] # subset containing preflop action until end
    for idx, elem in enumerate(subset):
        if '</round>' in elem:
            stop = idx
            break 
    subset = subset[:stop] # subset containing just preflop action   
    
    raise_happened = False # trigger for when a raise occures 
    count = 0
    for elem in subset:

        if 'Hero' in elem:
            break

        for sub_string in elem.split(): # gets amount shoved or raised & players name
            if 'player' in sub_string:
                player_name = sub_string.replace('player="', '').replace('"', '') 
            if 'sum' in sub_string:
                bet = float(sub_string.replace('sum="', '').replace('"', ''))            

        if ('type="23"' in elem or 'type="7"' in elem) and raise_happened == False: # finds shoves & raises, trying to avoid edge cases here but probably some extraneous stuff

            if 'type="7"' in elem:
                if bet > max(player_amount_bet.values()):# checks to see if shove was a raise or call 
                    raise_happened = True
            else: # for raises
                raise_happened = True

                
        if raise_happened:
            if 'type="3"' in elem:
                count += 1
            if 'type="7"' in elem:
                if bet <= max(player_amount_bet.values()):
                    count += 1
        if player_name not in player_amount_bet:
            player_amount_bet[player_name] = bet
        else:
            player_amount_bet[player_name] += bet
    return count 

In [56]:
df['callers'] = df['HandHistory'].apply(lambda x: callers(x))

In [58]:
df.callers.value_counts()

0    37832
1     1464
2      222
3       29
4        3
5        1
Name: callers, dtype: int64

In [39]:
df.limpers.iloc[955]


['<action no="10" player="P64_513163EY" type="0" sum="0" cards="" />',
 '<action no="11" player="P11_428501AV" type="0" sum="0" cards="" />',
 '<action no="12" player="P23_420145NW" type="3" sum="300" cards="" />',
 '<action no="13" player="Hero" type="0" sum="0" cards="" />',
 '<action no="14" player="P30_755134VL" type="3" sum="300" cards="" />',
 '<action no="15" player="P2_683833PE" type="0" sum="0" cards="" />',
 '<action no="16" player="P72_196053CF" type="4" sum="0" cards="" />']